NYC 311 request filtering using the 5 targeted weather stations, making this project a regional heat exposure proxy, *not* a micro-climate model.
Quality-ofLife (QoL) proxy variables are determined as follows:

- **qol_calls:** Count of 311 calls in QoL-related categories.
- **total_calls:** Count of all 311 calls in a week.
- **qol_pct:** qol_calls / total_calls * 100.
- **qol_rate_pc:** qol_calls / 1000 residents.

Timing is June through August, which is considered summer time for US.
Temporal resolution is by weeks.
Spatial resolution is in block groups.
Final resolution is 311 data aggregated to block groups × week and citywide weather data aggregated by week.

NYC 311 category selection for QoL have to consider the below:
- Meaningful livability impact reflecting disruptions or degradation of everyday life (rather than purely administrative or regulatory issues).
- Likely to respond to heat/stress, things that might get worse under high heat or strain (e.g., noise, sanitation, infrastructure).
- Sufficient volume and spatial distribution, common enough to show variation across block groups and days.
- Clear linkage to place and time, must have incident location, timestamp, etc.

**Heat and housing livability domain:**

- HEAT/HOT WATER
- UNSANITARY CONDITION
- GENERAL CONSTRUCTION
- PAINT/PLASTER
- ELECTRIC
- PLUMBING
- MOLD
- WATER LEAK
- BUILDING CONDITION
- UNSAFE BUILDING
- ELEVATOR

**Noise and outdoor comfort domain:**

- NOISE - RESIDENTIAL
- NOISE - STREET/SIDEWALK
- NOISE - VEHICLE
- NOISE - PARK

**Sanitation and public space domain:**

- SANITATION CONDITION
- DIRTY CONDITIONS
- LITTER BASKET / REQUEST
- ILLEGAL DUMPING
- GRAFFITI
- RAT SIGHTING / RODENT

**Street and infrastructure condition domain:**

- STREET CONDITION
- SIDEWALK CONDITION
- POTHOLE
- TRAFFIC SIGNAL CONDITION
- STREET LIGHT CONDITION
- BROKEN METER

**Environmental stressors:**

- SEWER
- FLOODING
- WATER SYSTEM
- CONSTRUCTION NOISE
- AIR QUALITY

**Parks, trees, public space:**

- TREE DAMAGE
- TREE DEBRIS
- DEAD TREE
- PARK GENERAL
- PLAYGROUND
- PARK MAINTENANCE & CLEANLINESS

**Neighborhood disorder:**

- ABANDONED VEHICLE
- DERELICT VEHICLE
- BLOCKED DRIVEWAY
- ILLEGAL PARKING

Block groups data link: https://catalog.data.gov/dataset/tiger-line-shapefile-current-state-new-york-block-group

311 data link: https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data

In [1]:
import pandas as pd
from pathlib import Path
import requests
import numpy as np
from pathlib import Path
from datetime import datetime
import time
import geopandas as gpd
from shapely.geometry import Point
import cenpy

In [2]:
# Load statewide 2020 block groups.
bg_path = "data/ny_block_groups_2020/ny_block_groups_2020.shp"
gdf_bg = gpd.read_file(bg_path)

# Filter to NYC counties only.
nyc_prefixes = ("36005", "36047", "36061", "36081", "36085")
gdf_bg = gdf_bg[gdf_bg["GEOID"].str.startswith(nyc_prefixes)].copy()

print("NYC block groups:", len(gdf_bg))

NYC block groups: 6807


In [3]:
base_url = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"
app_token = "vOli45DyYtVRbXMZ6YS2Amgjh"
page_size = 50000
years = range(2018, 2026)

select_cols = [
    "unique_key", "created_date", "complaint_type",
    "descriptor", "latitude", "longitude", "borough"
]

def download_311_summer_year(year):
    print(f"Downloading {year}…")

    start = f"{year}-06-01T00:00:00"
    end   = f"{year}-08-31T23:59:59"

    where = (
        f"created_date between '{start}' and '{end}' "
        "AND latitude IS NOT NULL AND longitude IS NOT NULL"
    )

    params = {
        "$select": ",".join(select_cols),
        "$where": where,
        "$limit": page_size,
        "$order": "created_date"
    }

    headers = {"X-App-Token": app_token}

    all_rows = []
    offset = 0

    while True:
        params["$offset"] = offset
        r = requests.get(base_url, params=params, headers=headers)

        if r.status_code != 200:
            print("Error:", r.status_code)
            break

        data = r.json()
        if not data:
            break

        df = pd.DataFrame(data)
        all_rows.append(df)

        print(f"  Retrieved {len(df)} at offset {offset}")
        offset += page_size
        time.sleep(0.2)

    if not all_rows:
        return pd.DataFrame()

    out = pd.concat(all_rows, ignore_index=True)
    out["created_date"] = pd.to_datetime(out["created_date"])
    out["year"] = out["created_date"].dt.year
    out["date"] = out["created_date"].dt.date

    out["latitude"] = pd.to_numeric(out["latitude"])
    out["longitude"] = pd.to_numeric(out["longitude"])

    return out

# Download all years
dfs = []
for y in years:
    df_y = download_311_summer_year(y)
    if not df_y.empty:
        dfs.append(df_y)

df_311 = pd.concat(dfs, ignore_index=True)
print("311 rows:", len(df_311))

  Retrieved 50000 at offset 0
  Retrieved 50000 at offset 50000
  Retrieved 50000 at offset 100000
  Retrieved 50000 at offset 150000
  Retrieved 50000 at offset 200000
  Retrieved 50000 at offset 250000
  Retrieved 50000 at offset 300000
  Retrieved 50000 at offset 350000
  Retrieved 50000 at offset 400000
  Retrieved 50000 at offset 450000
  Retrieved 50000 at offset 500000
  Retrieved 50000 at offset 550000
  Retrieved 50000 at offset 600000
  Retrieved 11902 at offset 650000
  Retrieved 50000 at offset 0
  Retrieved 50000 at offset 50000
  Retrieved 50000 at offset 100000
  Retrieved 50000 at offset 150000
  Retrieved 50000 at offset 200000
  Retrieved 50000 at offset 250000
  Retrieved 50000 at offset 300000
  Retrieved 50000 at offset 350000
  Retrieved 50000 at offset 400000
  Retrieved 50000 at offset 450000
  Retrieved 50000 at offset 500000
  Retrieved 50000 at offset 550000
  Retrieved 46963 at offset 600000
  Retrieved 50000 at offset 0
  Retrieved 50000 at offset 50000
  R

In [4]:
# Convert to GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df_311.longitude, df_311.latitude)]
gdf_311 = gpd.GeoDataFrame(df_311, geometry=geometry, crs="EPSG:4326")

# Reproject to match BG CRS
if gdf_311.crs != gdf_bg.crs:
    gdf_311 = gdf_311.to_crs(gdf_bg.crs)

# Spatial join
gdf_joined = gpd.sjoin(
    gdf_311,
    gdf_bg[["GEOID", "geometry"]],
    how="left",
    predicate="within"
)

gdf_joined = gdf_joined.dropna(subset=["GEOID"]).copy()

# Clean name
gdf_joined.rename(columns={"GEOID": "GEOID_BG"}, inplace=True)

In [5]:
df_panel = (
    gdf_joined
    .groupby(["GEOID_BG", "date"], as_index=False)
    .agg(
        total_calls=("unique_key", "count")
    )
)

In [6]:
QOL_TYPES = [
    "HEAT", "HOT WATER", "WATER LEAK", "PLUMBING", "GAS",
    "PAINT", "PLASTER", "ELECTRIC", "MOLD", "ELEVATOR",
    "UNSANITARY", "DIRTY", "SANITATION", "HOUSING", "UNSAFE BUILDING",
    "NOISE - RESIDENTIAL", "NOISE - PARK", "NOISE - STREET",
    "RODENT", "AIR QUALITY", "GRAFFITI", "SEWER", "FLOODING",
    "SIDEWALK", "POTHOLE", "TRAFFIC SIGNAL", "STREET LIGHT",
]

gdf_joined["qol_flag"] = gdf_joined["complaint_type"].str.upper().isin(QOL_TYPES)

qol_panel = (
    gdf_joined
    .groupby(["GEOID_BG", "date"])
    .agg(qol_calls=("qol_flag", "sum"))
    .reset_index()
)

df_panel = df_panel.merge(qol_panel, on=["GEOID_BG", "date"], how="left")
df_panel["qol_calls"] = df_panel["qol_calls"].fillna(0).astype(int)
df_panel["qol_pct"] = df_panel["qol_calls"] / df_panel["total_calls"]

In [7]:
heat = pd.read_csv("data/nyc_heat_exposure_2018_2025.csv")
heat["DATE"] = pd.to_datetime(heat["DATE"])

df_panel["date"] = pd.to_datetime(df_panel["date"])
panel = df_panel.merge(heat, left_on="date", right_on="DATE", how="left")
panel = panel.drop(columns=["DATE"])

In [8]:
api = cenpy.remote.APIConnection("ACSDT5Y2022")

acs_vars = [
    "NAME",
    "B01003_001E",  # total population
    "B19013_001E",  # median HH income
    "B17001_001E",  # poverty universe
    "B17001_002E",  # poverty count
    "B25044_003E"   # no-vehicle households
]

nyc_counties = ["005", "047", "061", "081", "085"]
acs_dfs = []

for county in nyc_counties:
    df = api.query(
        cols=acs_vars,
        geo_unit="block group",
        geo_filter={"state": "36", "county": county}
    )
    acs_dfs.append(df)

acs = pd.concat(acs_dfs, ignore_index=True)
acs["GEOID_BG"] = acs["state"] + acs["county"] + acs["tract"] + acs["block group"]

In [9]:
bad_codes = [-666666666, -888888888, -222222222, -333333333]

acs = acs.replace(bad_codes, np.nan)

# Convert numerics
for col in ["B01003_001E","B19013_001E","B17001_001E","B17001_002E","B25044_003E"]:
    acs[col] = pd.to_numeric(acs[col], errors="coerce")

acs.rename(columns={
    "B01003_001E":"pop_total",
    "B19013_001E":"medhhinc",
    "B17001_001E":"poverty_universe",
    "B17001_002E":"poverty_count",
    "B25044_003E":"no_vehicle_hh"
}, inplace=True)

In [10]:
panel = panel.merge(acs, on="GEOID_BG", how="left")

In [11]:
panel["poverty_rate"] = panel["poverty_count"] / panel["poverty_universe"]
panel["poverty_rate"] = panel.groupby("tract")["poverty_rate"].transform(lambda x: x.fillna(x.median()))
panel["poverty_rate"] = panel["poverty_rate"].fillna(panel["poverty_rate"].median())

In [12]:
panel["poverty_rate"] = panel["poverty_count"] / panel["poverty_universe"]
panel["poverty_rate"] = panel.groupby("tract")["poverty_rate"].transform(lambda x: x.fillna(x.median()))
panel["poverty_rate"] = panel["poverty_rate"].fillna(panel["poverty_rate"].median())

In [ ]:
panel["dow"] = panel["date"].dt.dayofweek
panel["year"] = panel["date"].dt.year
panel["log_total_calls"] = np.log(panel["total_calls"].replace(0,1))

In [14]:
# ===========================================================
# BUILD ALL REQUIRED MODEL VARIABLES BEFORE DROPPING NAs
# ===========================================================

# 1. Convert heat features
panel["tmax_city_f"] = panel["TMAX_CITY"]
panel["tmean_city_f"] = panel["TMEAN_CITY"]

# 2. Safe poverty rate
panel["poverty_rate"] = (
    panel["poverty_count"] / panel["poverty_universe"]
).replace([np.inf, -np.inf], np.nan)

# 3. Tract-level imputation
panel["poverty_rate"] = panel.groupby("tract")["poverty_rate"].transform(
    lambda x: x.fillna(x.median())
)

# 4. Citywide fallback
panel["poverty_rate"] = panel["poverty_rate"].fillna(panel["poverty_rate"].median())

# 5. Center SES variables
panel["poverty_rate_c"] = panel["poverty_rate"] - panel["poverty_rate"].mean()
panel["medhhinc_c"] = panel["medhhinc"] - panel["medhhinc"].mean()

# 6. Interactions
panel["extreme_x_poverty"] = panel["EXTREME_HEAT"] * panel["poverty_rate_c"]
panel["extreme_x_no_vehicle"] = panel["EXTREME_HEAT"] * (
    panel["no_vehicle_hh"] / panel["pop_total"]
)

# 7. Time variables
panel["dow"] = panel["date"].dt.dayofweek
panel["year"] = panel["date"].dt.year

# 8. Log offset
panel["log_total_calls"] = np.log(panel["total_calls"].replace(0,1))

In [15]:
panel_clean = panel.dropna(subset=[
    "qol_calls","total_calls","poverty_rate_c","medhhinc_c",
    "EXTREME_HEAT","tmax_city_f","extreme_x_poverty",
    "log_total_calls"
])

panel_clean.to_csv(
    "data/model/nyc311_heat_acs_bg_2018_2025_final_regression_cleaned.csv",
    index=False
)